In [20]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
weather_df = pd.read_csv("weather data.csv", usecols=["ZIP", "DATE", "TEMP MAX", "TEMP MIN", "TEMP AVG", "RAIN", "SNOW",
                        "WIND", "GUST", "TEMP NORM", "SOIL MOISTURE", "SOIL TEMP", "DMA ID"], dtype={"ZIP": "int32",
                        "TEMP MAX": "float32", "TEMP MIN": "float32", "TEMP AVG": "float32", "RAIN": "float32", "SNOW": "float32",
                        "WIND": "float32", "GUST": "float32", "TEMP NORM": "float32", "SOIL MOISTURE": "float32", "SOIL TEMP": "float32",
                        "DMA ID": "int16"}, parse_dates=True) # Source Central
sales_df = pd.read_csv("sales data.csv", dtype={"DMA ID": "int16", "DMA NAME": "category", "PRODUCT": "category",
                                              "UNITS": "float16", "DOLLARS": "float32", "NOTE": "category"}, parse_dates=True) # Source Central

# Note: Some ZIP codes have 0 population
pop_df = pd.read_csv("population data.csv", usecols=["POPULATION", "ZIP"], dtype={"POPULATION": "int32", "ZIP": "int32"}) # Source US Census

In [3]:
from pandas.tseries.holiday import USFederalHolidayCalendar
import datetime 
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2018-01-01', end='2023-12-31').to_pydatetime()


In [4]:
print(holidays)

[datetime.datetime(2018, 1, 1, 0, 0) datetime.datetime(2018, 1, 15, 0, 0)
 datetime.datetime(2018, 2, 19, 0, 0) datetime.datetime(2018, 5, 28, 0, 0)
 datetime.datetime(2018, 7, 4, 0, 0) datetime.datetime(2018, 9, 3, 0, 0)
 datetime.datetime(2018, 10, 8, 0, 0)
 datetime.datetime(2018, 11, 12, 0, 0)
 datetime.datetime(2018, 11, 22, 0, 0)
 datetime.datetime(2018, 12, 25, 0, 0) datetime.datetime(2019, 1, 1, 0, 0)
 datetime.datetime(2019, 1, 21, 0, 0) datetime.datetime(2019, 2, 18, 0, 0)
 datetime.datetime(2019, 5, 27, 0, 0) datetime.datetime(2019, 7, 4, 0, 0)
 datetime.datetime(2019, 9, 2, 0, 0) datetime.datetime(2019, 10, 14, 0, 0)
 datetime.datetime(2019, 11, 11, 0, 0)
 datetime.datetime(2019, 11, 28, 0, 0)
 datetime.datetime(2019, 12, 25, 0, 0) datetime.datetime(2020, 1, 1, 0, 0)
 datetime.datetime(2020, 1, 20, 0, 0) datetime.datetime(2020, 2, 17, 0, 0)
 datetime.datetime(2020, 5, 25, 0, 0) datetime.datetime(2020, 7, 3, 0, 0)
 datetime.datetime(2020, 9, 7, 0, 0) datetime.datetime(2020, 

In [5]:
# Parsing the dates to a date format
sales_df["DATE"] = pd.to_datetime(sales_df["DATE"])
weather_df["DATE"] = pd.to_datetime(weather_df["DATE"])

# Generate data based features
sales_df["YEAR"] = sales_df["DATE"].dt.strftime("%Y")
weather_df["YEAR"] = weather_df["DATE"].dt.strftime("%Y")
weather_df['Month'] = weather_df['DATE'].dt.strftime('%m').astype(float)
weather_df['WeekDay'] = weather_df['DATE'].dt.strftime('%w').astype(float)
weather_df['WeekNumber'] = weather_df['DATE'].dt.strftime('%V').astype(float) 
weather_df['DayOfYear'] = weather_df['DATE'].dt.strftime('%j').astype(float) 
weather_df['IsHoliday'] = weather_df['DATE'].isin(holidays).astype(int)

In [6]:
# Joining the weather and population data together
merged_weather = weather_df.merge(pop_df[["POPULATION", "ZIP"]], on="ZIP", how="left")
merged_weather["POPULATION"].fillna(0, inplace=True)
merged_weather.head()

,ZIP,DATE,TEMP MAX,TEMP MIN,TEMP AVG,RAIN,SNOW,WIND,GUST,TEMP NORM,SOIL MOISTURE,SOIL TEMP,DMA ID,YEAR,Month,WeekDay,WeekNumber,DayOfYear,IsHoliday,POPULATION
0,30032,2018-06-09,89.900002,67.800003,78.900002,0.03,0.0,2.0,6.0,76.900002,NaN,NaN,524,2018,6.0,6.0,23.0,160.0,0,44697.0
1,30026,2018-06-09,89.599998,66.800003,78.199997,0.05,0.0,2.0,6.0,76.599998,NaN,NaN,524,2018,6.0,6.0,23.0,160.0,0,0.0
2,30029,2018-06-09,89.699997,67.000000,78.300003,0.05,0.0,2.0,6.0,76.699997,NaN,NaN,524,2018,6.0,6.0,23.0,160.0,0,0.0
3,30095,2018-06-09,89.699997,67.000000,78.300003,0.05,0.0,2.0,6.0,76.699997,NaN,NaN,524,2018,6.0,6.0,23.0,160.0,0,0.0
4,30099,2018-06-09,89.699997,67.000000,78.300003,0.05,0.0,2.0,6.0,76.699997,NaN,NaN,524,2018,6.0,6.0,23.0,160.0,0,0.0


In [7]:
# Creating a Total Population column
merged_weather = merged_weather.merge(merged_weather[["DMA ID", "DATE", "POPULATION"]].rename(columns={"POPULATION": "TOTAL POPULATION"}).groupby(by=["DMA ID", "DATE"]).sum(), on=["DMA ID", "DATE"])
#merged_weather_df = merged_weather.merge(sales_df[["DATE", "DMA ID", "DOLLARS"]].groupby(by=["DATE", "DMA ID"], as_index=False).sum(), on=["DMA ID", "DATE"])
merged_weather.head()


,ZIP,DATE,TEMP MAX,TEMP MIN,TEMP AVG,RAIN,SNOW,WIND,GUST,TEMP NORM,...,SOIL TEMP,DMA ID,YEAR,Month,WeekDay,WeekNumber,DayOfYear,IsHoliday,POPULATION,TOTAL POPULATION
0,30032,2018-06-09,89.900002,67.800003,78.900002,0.03,0.0,2.0,6.0,76.900002,...,NaN,524,2018,6.0,6.0,23.0,160.0,0,44697.0,7235998.0
1,30026,2018-06-09,89.599998,66.800003,78.199997,0.05,0.0,2.0,6.0,76.599998,...,NaN,524,2018,6.0,6.0,23.0,160.0,0,0.0,7235998.0
2,30029,2018-06-09,89.699997,67.000000,78.300003,0.05,0.0,2.0,6.0,76.699997,...,NaN,524,2018,6.0,6.0,23.0,160.0,0,0.0,7235998.0
3,30095,2018-06-09,89.699997,67.000000,78.300003,0.05,0.0,2.0,6.0,76.699997,...,NaN,524,2018,6.0,6.0,23.0,160.0,0,0.0,7235998.0
4,30099,2018-06-09,89.699997,67.000000,78.300003,0.05,0.0,2.0,6.0,76.699997,...,NaN,524,2018,6.0,6.0,23.0,160.0,0,0.0,7235998.0


In [8]:
merged_weather_sales = merged_weather.merge(sales_df[["DATE", "DMA ID", "DOLLARS"]].groupby(by=["DATE", "DMA ID"], as_index=False).sum(), on=["DMA ID", "DATE"])
merged_weather_sales.head()

,ZIP,DATE,TEMP MAX,TEMP MIN,TEMP AVG,RAIN,SNOW,WIND,GUST,TEMP NORM,...,DMA ID,YEAR,Month,WeekDay,WeekNumber,DayOfYear,IsHoliday,POPULATION,TOTAL POPULATION,DOLLARS
0,30032,2018-06-09,89.900002,67.800003,78.900002,0.03,0.0,2.0,6.0,76.900002,...,524,2018,6.0,6.0,23.0,160.0,0,44697.0,7235998.0,5627.160156
1,30026,2018-06-09,89.599998,66.800003,78.199997,0.05,0.0,2.0,6.0,76.599998,...,524,2018,6.0,6.0,23.0,160.0,0,0.0,7235998.0,5627.160156
2,30029,2018-06-09,89.699997,67.000000,78.300003,0.05,0.0,2.0,6.0,76.699997,...,524,2018,6.0,6.0,23.0,160.0,0,0.0,7235998.0,5627.160156
3,30095,2018-06-09,89.699997,67.000000,78.300003,0.05,0.0,2.0,6.0,76.699997,...,524,2018,6.0,6.0,23.0,160.0,0,0.0,7235998.0,5627.160156
4,30099,2018-06-09,89.699997,67.000000,78.300003,0.05,0.0,2.0,6.0,76.699997,...,524,2018,6.0,6.0,23.0,160.0,0,0.0,7235998.0,5627.160156


In [21]:
# XG boost train and test using all data available in the dataset. 
# We are not including any features engineered outside of what is provided in the data set
import xgboost as xgb
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
startify_arr = merged_weather_sales['DMA ID']
X, y = merged_weather_sales[['ZIP','TEMP MAX', 'TEMP MIN', 'TEMP AVG','RAIN','SNOW','WIND',
                             'GUST','TEMP NORM', 'SOIL MOISTURE', 'SOIL TEMP','DMA ID']], merged_weather_sales['DOLLARS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=startify_arr)
#Creating an XGBoost regressor
model = xgb.XGBRegressor()

#Training the model on the training data
model.fit(X_train, y_train)

#Making predictions on the test set
predictions = model.predict(X_test)

# Calculate the mean squared error and R-squared score
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Mean Squared Error: 1683432.9
R-squared Score: 0.7452875576297342


In [22]:
merged_weather_sales_2022 = merged_weather_sales[merged_weather_sales['DATE']<= '2022-12-31']
merged_weather_sales_2023 = merged_weather_sales[merged_weather_sales['DATE']>= '2023-01-01']

In [23]:
# XG boost train and test using 2022 data. 
# We are not including any features engineered outside of what is provided in the data set
startify_arr = merged_weather_sales_2022['DMA ID']
X, y = merged_weather_sales_2022[['ZIP','TEMP MAX', 'TEMP MIN', 'TEMP AVG','RAIN','SNOW','WIND',
                             'GUST','TEMP NORM', 'SOIL MOISTURE', 'SOIL TEMP','DMA ID']], merged_weather_sales_2022['DOLLARS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=startify_arr)
#Creating an XGBoost regressor
model = xgb.XGBRegressor()

#Training the model on the training data
model.fit(X_train, y_train)

#Making predictions on the test set
predictions_test = model.predict(X_test)

# Calculate the mean squared error and R-squared score
mse = mean_squared_error(y_test, predictions_test)
r2 = r2_score(y_test, predictions_test)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Mean Squared Error: 1754719.8
R-squared Score: 0.7416372595582439


In [24]:
# Now validate using 2023 data that the model has not seen durng training.

X_validation,Y_Validation = merged_weather_sales_2023[['ZIP','TEMP MAX', 'TEMP MIN', 'TEMP AVG','RAIN','SNOW','WIND',
                             'GUST','TEMP NORM', 'SOIL MOISTURE', 'SOIL TEMP','DMA ID']], merged_weather_sales_2023['DOLLARS']
predictions_validation = model.predict(X_validation)
mse = mean_squared_error(Y_Validation, predictions_validation)
r2 = r2_score(Y_Validation, predictions_validation)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Mean Squared Error: 1637916.8
R-squared Score: 0.6817787118589199


In [25]:
# XG boost train and test using all data available in the dataset.
# We are including date based features that was extracted from the dataset
import xgboost as xgb
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
startify_arr = merged_weather_sales['DMA ID']
X, y = merged_weather_sales[['ZIP','TEMP MAX', 'TEMP MIN', 'TEMP AVG','Month','RAIN','SNOW','WIND',
                             'GUST','TEMP NORM', 'SOIL MOISTURE', 'SOIL TEMP','DMA ID','WeekDay','WeekNumber',
                             'DayOfYear', 'IsHoliday']], merged_weather_sales['DOLLARS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=startify_arr)
#Creating an XGBoost regressor
model = xgb.XGBRegressor()

#Training the model on the training data
model.fit(X_train, y_train)

#Making predictions on the test set
predictions = model.predict(X_test)

# Calculate the mean squared error and R-squared score
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Mean Squared Error: 839671.0
R-squared Score: 0.8729532936223652


In [26]:
# XG boost train and test using 2022 data. 
# We are including date based features that was extracted from the dataset
startify_arr = merged_weather_sales_2022['DMA ID']
X, y = merged_weather_sales_2022[['ZIP','TEMP MAX', 'TEMP MIN', 'TEMP AVG','Month','RAIN','SNOW','WIND',
                             'GUST','TEMP NORM', 'SOIL MOISTURE', 'SOIL TEMP','DMA ID','WeekDay','WeekNumber',
                             'DayOfYear', 'IsHoliday']], merged_weather_sales_2022['DOLLARS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=startify_arr)
#Creating an XGBoost regressor
model = xgb.XGBRegressor()

#Training the model on the training data
model.fit(X_train, y_train)

#Making predictions on the test set
predictions_test = model.predict(X_test)

# Calculate the mean squared error and R-squared score
mse = mean_squared_error(y_test, predictions_test)
r2 = r2_score(y_test, predictions_test)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Mean Squared Error: 843685.6
R-squared Score: 0.8757767930224802


In [27]:
# Now validate using 2023 data that the model has not seen durng training.
X_validation,Y_Validation = merged_weather_sales_2023[['ZIP','TEMP MAX', 'TEMP MIN', 'TEMP AVG','Month','RAIN','SNOW','WIND',
                             'GUST','TEMP NORM', 'SOIL MOISTURE', 'SOIL TEMP','DMA ID','WeekDay','WeekNumber',
                             'DayOfYear', 'IsHoliday']], merged_weather_sales_2023['DOLLARS']
predictions_validation = model.predict(X_validation)
mse = mean_squared_error(Y_Validation, predictions_validation)
r2 = r2_score(Y_Validation, predictions_validation)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Mean Squared Error: 1243587.5
R-squared Score: 0.7583906544311949
